<a href="https://colab.research.google.com/github/kutyadog/ai_notebooks/blob/main/llama_2_and_langchain_GP_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tests - Sept 2023

In [ ]:
# source
# https://github.com/curiousily/Get-Things-Done-with-Prompt-Engineering-and-LangChain/blob/master/13.chat-with-multiple-pdfs-using-llama-2-and-langchain.ipynb


!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off

In [ ]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [ ]:
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

In [ ]:
!sudo apt-get install poppler-utils

In [ ]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
# from pdf2image import convert_from_path
from langchain.document_loaders.csv_loader import CSVLoader
# from langchain.agents import create_csv_agent

from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Data

In [ ]:
# @title prep for all data
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)



In [ ]:
# @title load embeddings database
# !gdown 1v-Rn1FVU1pLTAQEgm0N9oB6cExMoebZr -O pdfs/tesla-earnings-report.pdf
# !gdown 1Xc890jrQvCExAkryVWAttsv1DBLdVefN -O pdfs/nvidia-earnings-report.pdf
# !gdown 1Epz-SQ3idPpoz75GlTzzomag8gplzLv8 -O pdfs/meta-earnings-report.pdf

# !gdown 1aK7p7ZlrX-QD-WWguBPUHfPX5WP-HBy1 -O formatted_articles.csv
!mkdir db
!gdown 1swt3Wt5l6_2cGdrWvjSnUKYw7OwmpAT3 -O db/chroma.sqlite3


Downloading...
From: https://drive.google.com/uc?id=1swt3Wt5l6_2cGdrWvjSnUKYw7OwmpAT3
To: /content/db/chroma.sqlite3
100% 20.9M/20.9M [00:00<00:00, 77.8MB/s]


In [ ]:
# @title Redo embeddings by loading question csv
# meta_images = convert_from_path("pdfs/meta-earnings-report.pdf", dpi=88)
# meta_images = convert_from_path("/content/formatted_articles.csv", dpi=88)
# meta_images[0]

loader = CSVLoader(file_path="formatted_articles.csv")
documents = loader.load()

In [ ]:
# nvidia_images = convert_from_path("pdfs/nvidia-earnings-report.pdf", dpi=88)
# nvidia_images[0]

In [ ]:
# tesla_images = convert_from_path("pdfs/tesla-earnings-report.pdf", dpi=88)
# tesla_images[0]

In [ ]:
# !rm -rf "db"

In [ ]:
# loader = PyPDFDirectoryLoader("pdfs")
# docs = loader.load()
# len(docs)

100

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(documents)
len(texts)

2934

In [ ]:
# This takes About 4 minutes - best to find a way to save it as csv
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 3min 58s, sys: 3.82 s, total: 4min 2s
Wall time: 4min 38s


In [ ]:
# load uploaded db
db = Chroma(persist_directory='./db/chroma', embedding_function=embeddings)

# test it out to find stuffs
docs = db.similarity_search("how can I change my 401k?")
print(docs)

[]


## Llama 2 13B

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

In [ ]:
!nvidia-smi

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [ ]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

## Chat with Multiple PDFs

In [ ]:
result = qa_chain("When was george washington born?")

 Based on the provided context, I cannot answer the question as there is no mention of George Washington's birthdate or any other relevant information about his life. The text only mentions the name "George Washington" and provides contact information for the Washington Post. Therefore, I do not have enough information to provide a correct answer to the question.


In [ ]:
len(result["source_documents"])
print( result["source_documents"])

[Document(page_content='context: You can make changes to your 401(k) at any time. <b>Changing your savings rate</b> Go to&nbsp;<a href="https://guidepost.washpost.com/pf/home">GuidePost&nbsp;</a>&gt; Okta apps &gt;&nbsp;<a href="https://washpost.okta.com/home/thewashingtonpostandcompaniesprod_vanguard_3/0oa1e7vb7q8XtIIUx0h8/aln1e7vk9ba4Tgt050h8">Vanguard</a>. Once you are logged in to your account in our plan, select MANAGE MY MONEY &gt; Change My Paycheck Deduction. Please confirm paycheck deductions from your payslip on or after the effective date. <b>Adjusting your investments</b> To change how your future contributions are invested, go to <a href="https://guidepost.washpost.com/pf/home">GuidePost&nbsp;</a>&gt; Okta apps &gt; <a href="https://washpost.okta.com/home/thewashingtonpostandcompaniesprod_vanguard_3/0oa1e7vb7q8XtIIUx0h8/aln1e7vk9ba4Tgt050h8">Vanguard</a>. Once you’re logged in select MANAGE MY MONEY &gt; Change my investments &gt; Change paycheck investment mix. Once you c

In [ ]:
print(result["source_documents"][0].page_content)

context: You can make changes to your 401(k) at any time. <b>Changing your savings rate</b> Go to&nbsp;<a href="https://guidepost.washpost.com/pf/home">GuidePost&nbsp;</a>&gt; Okta apps &gt;&nbsp;<a href="https://washpost.okta.com/home/thewashingtonpostandcompaniesprod_vanguard_3/0oa1e7vb7q8XtIIUx0h8/aln1e7vk9ba4Tgt050h8">Vanguard</a>. Once you are logged in to your account in our plan, select MANAGE MY MONEY &gt; Change My Paycheck Deduction. Please confirm paycheck deductions from your payslip on or after the effective date. <b>Adjusting your investments</b> To change how your future contributions are invested, go to <a href="https://guidepost.washpost.com/pf/home">GuidePost&nbsp;</a>&gt; Okta apps &gt; <a href="https://washpost.okta.com/home/thewashingtonpostandcompaniesprod_vanguard_3/0oa1e7vb7q8XtIIUx0h8/aln1e7vk9ba4Tgt050h8">Vanguard</a>. Once you’re logged in select MANAGE MY MONEY &gt; Change my investments &gt; Change paycheck investment mix. Once you complete this step, you’l

In [ ]:
# result = qa_chain("What is the per share revenue for Tesla during 2023?")

 I can't determine the per share revenue for Tesla in 2023 based on the information provided. The financial statements only provide data on total revenues and not per share data. Additionally, there is no information on the number of outstanding shares, which is necessary to calculate per share data.


In [ ]:
# result = qa_chain("What is the per share revenue for Nvidia during 2023?")

 Based on the information provided, the per share revenue for Nvidia during 2023 was $0.83. This can be calculated by dividing the total revenue for the period ($7,192 million) by the weighted average number of shares outstanding (basic) during the period (2,470 million).


In [ ]:
# print(result["source_documents"][1].page_content)

In [ ]:
# result = qa_chain("What is the estimated YOY revenue for Meta during 2023?")

 Based on the information provided in the press release, Meta's estimated YOY revenue for 2023 is expected to be between $32-34.5 billion, which represents an increase of 11-16% over 2022.


In [ ]:
# result = qa_chain("What is the estimated YOY revenue for Tesla during 2023?")

In [ ]:
# result = qa_chain("What is the estimated YOY revenue for Nvidia during 2023?")

 Based on the information provided, I cannot estimate the exact year-over-year (YOY) revenue for Nvidia during 2023. The condensed consolidated financial statements only provide information up to the first quarter of fiscal year 2024, which ended on April 30, 2023. There are three more quarters left in fiscal year 2023, and we do not have information on the remaining quarters. Therefore, I cannot estimate the full-year YOY revenue for Nvidia during 2023.


In [ ]:
# result = qa_chain(
#     "Which company is more profitable during 2023 Meta, Nvidia or Tesla and why?"
# )

 I cannot determine which company is more profitable during 2023 between Meta, Nvidia, and Tesla based on the information provided. The passage only discusses Nvidia's financial condition and results of operations, but does not provide any information about Meta or Tesla's financial performance during 2023. Additionally, the passage notes that investors should carefully consider the risk factors set forth in Nvidia's filings with the SEC before making any investment decisions, which suggests that the profitability of these companies may be subject to various factors and uncertainties. Therefore, without access to the financial information of all three companies, it is impossible to accurately compare their profitability during 2023.


In [ ]:
# result = qa_chain(
#     "Choose one company to invest (Tesla, Nvidia or Meta) to maximize your profits for the long term (10+ years)?"
# )

 I cannot provide a definitive answer to this question, as it is not appropriate for me to give financial advice or make recommendations on specific investments. However, based on the information provided in the context, NVIDIA appears to be a strong company with a diverse range of businesses and a history of innovation, including the development of GPU architecture for various applications such as scientific computing, AI, data science, and more. Additionally, the company has a solid track record of growth and profitability, and has been successful in transitioning to new products and business models over time. These factors could potentially contribute to long-term success and profitability for investors. However, it is important to conduct thorough research and consider multiple factors before making any investment decisions.


## References

- [Tesla Quarterly Report (Jul 21, 2023)](https://ir.tesla.com/_flysystem/s3/sec/000095017023033872/tsla-20230630-gen.pdf)
- [Meta Q2 2023 Earnings (Jul 26, 2023)](https://s21.q4cdn.com/399680738/files/doc_financials/2023/q2/Meta-06-30-2023-Exhibit-99-1-FINAL.pdf)
- [Nvidia Fiscal Q1 2024](https://s201.q4cdn.com/141608511/files/doc_financials/2024/q1/ecefb2b2-efcb-45f3-b72b-212d90fcd873.pdf)